---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# University towns, mean housing prices and recessions


Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    
    u_t=pd.read_csv('university_towns.txt', sep="\n", header = None)
    df = pd.DataFrame(columns=['State', 'RegionName'])
    for row in u_t.index:
        if (u_t.iloc[row][0])[-6:]=='[edit]':
            name_State=(u_t.iloc[row][0])[:-6]
        else:
            if "(" in (u_t.iloc[row][0]): 
                df= df.append({'State': name_State, 'RegionName': u_t.iloc[row][0].partition(" (")[0]}, ignore_index=True)               
            else:
                df= df.append({'State': name_State, 'RegionName': u_t.iloc[row][0]}, ignore_index=True)               
    return df
print(get_list_of_university_towns())
#print(get_list_of_university_towns().set_index(['State','RegionName']).index)
#get_list_of_university_towns()

In [3]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df=pd.read_excel('gdplev.xls', header=219, skipfooter=0,parse_cols=[4,5],names=['Quarter', 'GDP'])
    #print(df)
    for index in df.index[:-2]:
        if (df.loc[index,'GDP']>df.loc[index+1,'GDP']) and (df.loc[index+1,'GDP']>df.loc[index+2,'GDP']):
            start=df.loc[index,'Quarter']
            break   
    return start
#print(get_recession_start())
#get_recession_start()

In [4]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df=pd.read_excel('gdplev.xls', header=219, skipfooter=0,parse_cols=[4,5],names=['Quarter', 'GDP'])
    for index1 in df.index[:-2]:
        if (df.loc[index1,'GDP']>df.loc[index1+1,'GDP']) and (df.loc[index1+1,'GDP']>df.loc[index1+2,'GDP']):
            for index2 in df.index[:-2]:
                if (df.loc[index1+index2,'GDP']<df.loc[index1+index2+1,'GDP']) and (df.loc[index1+index2+1,'GDP']<df.loc[index1+index2+2,'GDP']):
                    start=df.loc[index1,'Quarter']
                    end=df.loc[index1+index2+2,'Quarter']          
                    break 
            break
    return end
#print(get_recession_end())

In [5]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df=pd.read_excel('gdplev.xls', header=219, skipfooter=0,parse_cols=[4,5],names=['Quarter', 'GDP'])
    for index1 in df.index[:-2]:
        if (df.loc[index1,'GDP']>df.loc[index1+1,'GDP']) and (df.loc[index1+1,'GDP']>df.loc[index1+2,'GDP']):
            for index2 in df.index[:-2]:
                if (df.loc[index1+index2,'GDP']<df.loc[index1+index2+1,'GDP']) and (df.loc[index1+index2+1,'GDP']<df.loc[index1+index2+2,'GDP']):
                    start=df.loc[index1,'Quarter']
                    end=df.loc[index1+index2+2,'Quarter']          
                    break 
            break
    #print(df.iloc[index1:index1+index2+2+1])
    return df.loc[df.iloc[index1:index1+index2+2+1]['GDP'].argmin(),'Quarter']
#print(get_recession_bottom())

In [6]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df=pd.read_csv('City_Zhvi_AllHomes.csv', sep=",", header = 0)
    df.replace({"State": states}, inplace=True)
    df.sort_values(by=['State'], inplace=True)
    df.set_index(["State","RegionName"], inplace=True)
    #print(df.loc['Delaware'])
    
    df.drop(df.columns[0:49], axis=1, inplace=True)
    df.columns = pd.to_datetime(df.columns)
    res = df.resample('Q', axis=1).mean()
    res = res.rename(columns=lambda col: '{}q{}'.format(col.year, col.quarter))
    return res
#print(convert_housing_data_to_quarters())

In [1]:
#import matplotlib.pyplot as plt
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. '''
    
    uni_town=convert_housing_data_to_quarters().loc[convert_housing_data_to_quarters().index.isin(get_list_of_university_towns().set_index(['State','RegionName']).index)]
    non_uni_town=convert_housing_data_to_quarters().loc[convert_housing_data_to_quarters().index.difference(get_list_of_university_towns().set_index(['State','RegionName']).index)]
    
    uni_town_recession=uni_town.loc[:,get_recession_start():get_recession_bottom()].mean()
    non_uni_town_recession=non_uni_town.loc[:,get_recession_start():get_recession_bottom()].mean()
    #print(uni_town.loc[:,get_recession_start():get_recession_bottom()].mean())
    #print(non_uni_town.loc[:,get_recession_start():get_recession_bottom()].mean())
    
    #figure1 = plt.figure()
    #plt.title(r'Evolution of housing prices during recession', fontsize=20)   
    #plt.plot(np.arange(len(non_uni_town_recession.index)),non_uni_town_recession.iloc[:]);
    #plt.plot(np.arange(len(uni_town_recession.index)),uni_town_recession.iloc[:]);
    #plt.xticks(np.arange(len(non_uni_town_recession.index)),uni_town_recession.index)
    #plt.yticks([190000, 200000, 210000, 220000, 230000,240000, 250000])
    #plt.ylim([185000, 255000])
    #plt.ylabel(r'Housing prices', fontsize=15)
    #plt.xlabel(r'Time', fontsize=15)
    #plt.legend([r'Non-university towns', r'University towns'], loc='upper right', fontsize=13)
    #plt.show()
    
    #
    before_recession=list(uni_town).index(get_recession_start())-1
    uni_town['price_ratio'] = uni_town.apply (lambda row: row.loc[get_recession_bottom()]/row.iloc[before_recession], axis=1)
    non_uni_town['price_ratio'] = non_uni_town.apply (lambda row: row.loc[get_recession_bottom()]/row.iloc[before_recession], axis=1)
    
    print(uni_town['price_ratio'])
    print(non_uni_town['price_ratio'])
    #print(scipy.stats.ttest_ind(uni_town['price_ratio'],non_uni_town['price_ratio']))
    stats,p=scipy.stats.ttest_ind(uni_town['price_ratio'],non_uni_town['price_ratio'])
    #mean_price_uni_town=uni_town['price_ratio'].mean()
    #mean_price_non_uni_town=non_uni_town['price_ratio'].mean()
    
    if mean_price_uni_town<mean_price_non_uni_town:
        better="university town"
    else:
        better="non-university town"
        
    if p<0.01:
        different=True
    else:
        different=False

    '''Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    return (different, p, better)
    
#print(run_ttest())